In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as pdp

In [2]:
train = pd.read_csv('../home-credit-default-risk/application_train.csv')
test = pd.read_csv('../home-credit-default-risk/application_test.csv')
bureau = pd.read_csv('../home-credit-default-risk/bureau.csv')
bureau_b = pd.read_csv('../home-credit-default-risk/bureau_balance.csv')
prep = pd.read_csv('../home-credit-default-risk/previous_application.csv')

## application_train.csv

In [3]:
print('train.shape: ', train.shape)
train.head()

train.shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#missingData percentage
hig_mis_columns = train.loc[:,train.isnull().sum()/len(train) > 0.60].columns
hig_mis_columns

Index(['OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'FLOORSMIN_AVG',
       'LIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'FLOORSMIN_MODE', 'LIVINGAPARTMENTS_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'FLOORSMIN_MEDI', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
       'FONDKAPREMONT_MODE'],
      dtype='object')

以上が60%以上の欠損値カラムを削除する

In [5]:
new_train = train.drop(hig_mis_columns, axis=1)
new_train.shape

(307511, 105)

In [6]:
new_test = test.drop(hig_mis_columns, axis=1)
new_test.shape

(48744, 104)

## merge bureau.csv & application_train.csv

In [7]:
print('train: ', new_train.shape)
print('bureau: ', bureau.shape)

train:  (307511, 105)
bureau:  (1716428, 17)


trainのデータ数よりbureauのデータ数の方が少ない<br>
=　各SK_ID_CURRにbureauデータが必ずあるわけではない

In [8]:
train_with_bureau = train.loc[train['SK_ID_CURR'].isin(bureau['SK_ID_CURR']), :]
print('train with bureau count: ', len(train_with_bureau))

train with bureau count:  263491


In [9]:
test_with_bureau = test.loc[test['SK_ID_CURR'].isin(bureau['SK_ID_CURR']), :]
print('test with bureau count: ', len(test_with_bureau))

test with bureau count:  42320


In [10]:
print('bureau all: ', len(train_with_bureau)+len(test_with_bureau))

bureau all:  305811


In [11]:
train_bu = pd.merge(new_train, bureau, on='SK_ID_CURR', how ='inner')
test_bu = pd.merge(new_test, bureau, on='SK_ID_CURR', how ='inner')

## merge bureau_balance.csv

In [12]:
bureau_b.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [13]:
print('len train_bu : ', len(train_bu))
print('len test_bu : ', len(test_bu))
print('len bureau_balance: ', len(bureau_b))

len train_bu :  1465325
len test_bu :  251103
len bureau_balance:  27299925


In [14]:
train = pd.merge(train_bu, bureau_b, on='SK_ID_BUREAU', how ='inner')
test = pd.merge(test_bu, bureau_b, on='SK_ID_BUREAU', how ='inner')

## bureau data EDA

### missing value

In [16]:
train.isnull().sum()

SK_ID_CURR                           0
TARGET                               0
NAME_CONTRACT_TYPE                   0
CODE_GENDER                          0
FLAG_OWN_CAR                         0
FLAG_OWN_REALTY                      0
CNT_CHILDREN                         0
AMT_INCOME_TOTAL                     0
AMT_CREDIT                           0
AMT_ANNUITY_x                     1029
AMT_GOODS_PRICE                    617
NAME_TYPE_SUITE                  35675
NAME_INCOME_TYPE                     0
NAME_EDUCATION_TYPE                  0
NAME_FAMILY_STATUS                   0
NAME_HOUSING_TYPE                    0
REGION_POPULATION_RELATIVE           0
DAYS_BIRTH                           0
DAYS_EMPLOYED                        0
DAYS_REGISTRATION                    0
DAYS_ID_PUBLISH                      0
FLAG_MOBIL                           0
FLAG_EMP_PHONE                       0
FLAG_WORK_PHONE                      0
FLAG_CONT_MOBILE                     0
FLAG_PHONE               

ランダムフォレストは、欠損値を含んだデータも分析できるため、今回はそのまま。

### checking column content

In [17]:
profile = pdp.ProfileReport(train)
profile.to_file(outputfile="bureau_data_profile.html")